### Datacamp: Introduction to Deep Learning with Pytorch

Completed 6/10/2022

In [3]:
import torch
import torchvision
import torch.utils.data
import torchvision.transforms as transforms

In [10]:
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5), ((0.5)))])

# Prepare training set and testing set
trainset = torchvision.datasets.MNIST('mnist', train=True, 
                                      download=True, transform=transform)
testset = torchvision.datasets.MNIST('mnist', train=False,
                                     download=True, transform=transform)

# Prepare training loader and testing loader
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32,
                                          shuffle=True, num_workers=0)
testloader = torch.utils.data.DataLoader(testset, batch_size=32,
                                          shuffle=False, num_workers=0) 

In [11]:
import torch.nn as nn

In [12]:
class Net(nn.Module):
    def __init__(self, num_classes):
        super(Net, self).__init__()

        # Instantiate the ReLU nonlinearity
        self.relu = nn.ReLU()
        
        # Instantiate two convolutional layers
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=5, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(in_channels=5, out_channels=10, kernel_size=3, padding=1)
        
        # Instantiate a max pooling layer
        self.pool = nn.MaxPool2d(2, 2)
        
        # Instantiate a fully connected layer
        self.fc = nn.Linear(7 * 7 * 10, 10)

    def forward(self, x):
  
        # Apply conv followd by relu, then in next line pool
        x = self.relu(self.conv1(x))
        x = self.pool(x)

        # Apply conv followd by relu, then in next line pool
        x = self.relu(self.conv2(x))
        x = self.pool(x)

        # Prepare the image for the fully connected layer
        # USE THE SAME DIMENSIONAL INPUT AS DEFINED FOR THE FULLY-CONNECTED LAYER IN INIT (above)
        x = x.view(-1, 7 * 7 * 10)

        # Apply the fully connected layer and return the result
        return self.fc(x)

In [76]:
import torch.optim as optim

net = Net(10)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=3e-4)

for epoch in range(5):
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        optimizer.zero_grad()

        # Compute the forward pass
        outputs = net(inputs)
        
        # Compute the loss function
        loss = criterion(outputs, labels)
        
        # Compute the gradients
        loss.backward()
    
        # Update the weights
        optimizer.step()

In [77]:
correct, total = 0, 0
net.eval()

for i, data in enumerate(testloader, 0):
    inputs, labels = data
    total = labels.size(0)
    outputs = net(inputs)
    _, predicted = torch.max(outputs.data, 1)

for i, (p, l) in enumerate(zip(predicted, labels)):
    if p == l:
       correct += 1

            
print('The testing set accuracy is %d %%' % (100*correct/total))

The testing set accuracy is 100 %


In [78]:
print(labels)

tensor([1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 1, 2, 3, 4, 5, 6])


In [79]:
print(predicted)

tensor([1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 1, 2, 3, 4, 5, 6])


### Sequential Module

In [80]:
class Net2(nn.Module):
    def __init__(self):
        super(Net2, self).__init__()
        
        # Declare all the layers for feature extraction
        self.features = nn.Sequential(nn.Conv2d(in_channels=1, out_channels=5, kernel_size=3, padding=1), 
                                      nn.ReLU(inplace=True),
                                      nn.Conv2d(in_channels=5, out_channels=10, kernel_size=3, padding=1), 
                                      nn.MaxPool2d(2, 2), 
                                      nn.ReLU(inplace=True),
                                      nn.Conv2d(in_channels=10, out_channels=20, kernel_size=3, padding=1),
                                      nn.ReLU(inplace=True),
                                      nn.Conv2d(in_channels=20, out_channels=40, kernel_size=3, padding=1),
                                      nn.MaxPool2d(2, 2), 
                                      nn.ReLU(inplace=True))
        
        # Declare all the layers for classification
        self.classifier = nn.Sequential(nn.Linear(7 * 7 * 40, 1024), 
                                        nn.ReLU(inplace=True),
                                       	nn.Linear(1024, 2048), 
                                        nn.ReLU(inplace=True),
                                        nn.Linear(2048, 10))
        
    def forward(self, x):
      
        # Apply the feature extractor in the input
        x = self.features(x)
        
        # Squeeze the three spatial dimensions in one
        x = x.view(-1, 7 * 7 * 40)
        
        # Classify the images
        x = self.classifier(x)
        return x

In [84]:
import numpy as np
indices = np.arange(0, 600000, 1)

In [85]:
indices

array([     0,      1,      2, ..., 599997, 599998, 599999])

### Regularization Example

In [87]:
# Instantiate the network
model = Net(10)

# Instantiate the cross-entropy loss
criterion = nn.CrossEntropyLoss()

# Instantiate the Adam optimizer with Ridge lambda=0.001
optimizer = optim.Adam(model.parameters(), lr=3e-4, weight_decay=0.001)

### Dropout Example

Typically applied to fully-connected layers

In [88]:
class DropoutNet(nn.Module):
    def __init__(self):
        
        # Define all the parameters of the net
        self.classifier = nn.Sequential(
            nn.Linear(28*28, 200),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5),
            nn.Linear(200, 500),
            nn.ReLU(inplace=True),
            nn.Linear(500, 10))
        
    def forward(self, x):
    
    	# Do the forward pass
        return self.classifier(x)

### Batch-normalization Example

Used to make CNN training more efficient, while having regularization effects

In [89]:
class BNNet(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        # Implement the sequential module for feature extraction
        self.features = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=10, kernel_size=3, stride=1, padding=1),
            nn.MaxPool2d(2, 2), 
            nn.ReLU(inplace=True), 
            nn.BatchNorm2d(10),
            nn.Conv2d(in_channels=10, out_channels=20, kernel_size=3, stride=1, padding=1),
            nn.MaxPool2d(2, 2), 
            nn.ReLU(inplace=True), 
            nn.BatchNorm2d(20))
        
        # Implement the fully connected layer for classification (starting image size was 28x28)
        self.fc = nn.Linear(in_features=7 * 7 * 20, out_features=10)

### Loading a Previously Trained Model

"Fine Tuning Technique"

The previously trained model was used to classify digits; in this example it is repurposed for classifying letters

In [95]:
# Create a model using
alpha_model = Net(10)

# Load the parameters from the old model (doesn't actually exist)
# alpha_model.load_state_dict(torch.load('my_net.pth'))

# Change the number of out channels (26 - for 26 letters in the alphabet)
alpha_model.fc = nn.Linear(7 * 7 * 512, 26)

# Train and evaluate the model
alpha_model.train()
# train_net(alphamodel, optimizer, criterion)
print("Accuracy of the net is: " + str(alpha_model.eval()))

Accuracy of the net is: Net(
  (relu): ReLU()
  (conv1): Conv2d(1, 5, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(5, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc): Linear(in_features=25088, out_features=26, bias=True)
)


### Example of using a Pretrained Model

In [96]:
# Import the module
import torchvision

# Download resnet18
model = torchvision.models.resnet18(pretrained=True)

# Freeze all the layers bar the last one
for param in model.parameters():
    param.requires_grad = False

# Change the number of output units
model.fc = nn.Linear(512, 7)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to C:\Users\krash/.cache\torch\hub\checkpoints\resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]